# Numerical Simulation of Ballistic Performance of Honeycomb Sandwich Panels

### Objective:
Setup a parametric model in PyMAPDL for a honeycomb sandwich panel. Use it for running a Ballistic impact simulation

The learning objectives of this demo are:
* Setup and solve a parametric model using PyMAPDL
* Interactive plotting of CAD, mesh, and results in Pythonic interface.

### Model Details

#### Model parameters:
* Cell size
* Cell wall thickness
* Node length
* Facesheet thickness

## Step 1 - define all parameters

In [1]:
## Main parameters
cell_size = 0.006
cell_wall_thickness = 0.002
node_length = 0.004
facesheet_thickness = 0.005

In [2]:
# Fixed constants
structure_length = 0.1
structure_breadth = 0.1

In [3]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt

# Additional calculation 
L = cell_size/ np.sqrt(3)
offset_x = cell_size + cell_wall_thickness
offset_y = 1.5*L + cell_wall_thickness*(np.sin(np.pi/3))
Nx = int(structure_length / offset_x) + 1
Ny = int(structure_breadth / offset_y) + 1
print("L = ", L)
print("offset_x = ", offset_x)
print("offset_y = ", offset_y)
print("Nx = ", Nx)
print("Ny = ", Ny)

L =  0.0034641016151377548
offset_x =  0.008
offset_y =  0.0069282032302755096
Nx =  13
Ny =  15


# Step 2 - launch MAPDL and create geometry

In [4]:
from ansys.mapdl.core import launch_mapdl

# start mapdl
mapdl = launch_mapdl()
print(mapdl)

PyMAPDL is taking longer than expected to connect to an MAPDL session.
Checking if there are any available licenses...
Product:             Ansys Mechanical Enterprise
MAPDL Version:       23.1
ansys.mapdl Version: 0.64.0



In [5]:
## reset mapdl
mapdl.clear()

## enter the preprocessor
mapdl.prep7()

mapdl.units('SI')      # SI unit system

## Create geometry
mapdl.block(0, structure_length, 0, structure_breadth, 0, node_length)

plate = mapdl.cm("PLATE", "VOLU")

mapdl.clocal( 11, 0, structure_length/2, structure_breadth/2, 0)                                         ## Making local coordinate system
mapdl.wpcsys('', 11,)                                                                                    ## Shift working plane

initial_distance = node_length + 0.02                                              ## 20 mm from center of front facesheet of sandwich panel

# 0.44 Remington Magnum Bullet
mapdl.cone(0.00545, 0.005486, initial_distance , initial_distance + 0.00825)
mapdl.cone(0.00569, 0.005805, initial_distance + 0.00825 , initial_distance + 0.04089 )
mapdl.vadd(1,2)
bullet = mapdl.cm("BULLET","VOLU")

mapdl.csdele(11)                                                                   ## Delete local coordinate system
# mapdl.input('bullet9.anf')

mapdl.vplot('all')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## Step 3 - define material properties

In [6]:
# Define material properties
mapdl.mp("EX", 1, 2e5)  # Youngs modulus
mapdl.mp("PRXY", 1, 0.3367)  # Poissons ratio

mapdl.tb("EVISC",1)

## Define element attributes
mapdl.et(1, "SOLID185") # 3D 8-Node Layered Solid

1

In [7]:
# Saves the geometry in IGES format
mapdl.cdwrite("ALL",fname="C:/Users/HPRVa/Ballistic_shield_design_optimization/src/ geom",ext='igs')

*** MAPDL GLOBAL STATUS *** 

 TITLE =                                                                       
        24 KEYPOINTS DEFINED     MAX KEYPOINT NUMBER =             48
        32 LINES DEFINED         MAX LINE NUMBER =                 64
        14 AREAS DEFINED         MAX AREA NUMBER =                 28
         3 VOLUMES DEFINED       MAX VOLUME NUMBER =                3
         1 ELEM TYPES DEFINED    MAX ELEM TYPE NUMBER =             1
         0 ELEMENTS DEFINED      MAX ELEMENT NUMBER =               0
         0 NODES DEFINED         MAX NODE NUMBER =                  0

 WRITE ANSYS DATABASE AS AN ANSYS INPUT FILE: C:/Users/HPRVa/Ballistic_shield_design_optimization/src/ geom.igs                                                                                                                                                                                                   

 WRITE IGES FILE= C:/Users/HPRVa/Ballistic_shield_design_optimization/src/

 ATTRIBUTES WIL

In [8]:
mapdl.exit()

In [9]:
import json
import os
import grpc

from ansys.mechanical.core import launch_mechanical

mechanical = launch_mechanical(batch=True, cleanup_on_exit=False)
print(mechanical)

Ansys Mechanical [Ansys Mechanical Enterprise]
Product Version:231
Software build date:Wednesday, November 23, 2022 6:49:22 PM


In [10]:
geometry_path = "C:/Users/HPRVa/Ballistic_shield_design_optimization/src/geom.iges"

In [12]:
# project_directory = mechanical.project_directory
project_directory = "C:/Users/HPRVa/Ballistic_shield_design_optimization/src"
print(f"project directory = {project_directory}")

# Upload the file to the project directory.
mechanical.upload(file_name=geometry_path, file_location_destination=project_directory)

# Build the path relative to project directory.
base_name = os.path.basename(geometry_path)
combined_path = os.path.join(project_directory, base_name)
part_file_path = combined_path.replace("\\", "\\\\")
mechanical.run_python_script(f"part_file_path='{part_file_path}'")

# Verify the path
result = mechanical.run_python_script("part_file_path")
print(f"part_file_path on server: {result}")

project directory = C:/Users/HPRVa/Ballistic_shield_design_optimization/src


FileNotFoundError: Unable to locate filename C:/Users/HPRVa/Ballistic_shield_design_optimization/src/geom.iges.

In [ ]:
output = mechanical.run_python_script(
    """
import json

geometry_import_group_11 = Model.GeometryImportGroup
geometry_import_19 = geometry_import_group_11.AddGeometryImport()

geometry_import_19_format = Ansys.Mechanical.DataModel.Enums.GeometryImportPreference.\
    Format.Automatic
geometry_import_19_preferences = Ansys.ACT.Mechanical.Utilities.GeometryImportPreferences()
geometry_import_19_preferences.ProcessNamedSelections = True
geometry_import_19_preferences.ProcessCoordinateSystems = True

geometry_import_19.Import(part_file_path, geometry_import_19_format, geometry_import_19_preferences)

# #region Context Menu Action
# mesh_15 = Model.Mesh
# mesh_15.GenerateMesh()
# #endregion

# #region Toolbar Action
# model_11 = Model
# analysis_36 = model_11.AddExplicitDynamicsAnalysis()
# #endregion

# Model.AddStaticStructuralAnalysis()
# STAT_STRUC = Model.Analyses[0]
# CS_GRP = Model.CoordinateSystems
# ANALYSIS_SETTINGS = STAT_STRUC.Children[0]
# SOLN= STAT_STRUC.Solution

##### Section 2 Set up the Unit System.

ExtAPI.Application.ActiveUnitSystem = MechanicalUnitSystem.StandardMKS
ExtAPI.Application.ActiveAngleUnit = AngleUnitType.Radian

##### Section 3 Named Selection and Coordinate System.

NS1 = Model.NamedSelections.Children[0]
NS2 = Model.NamedSelections.Children[1]
NS3 = Model.NamedSelections.Children[2]
NS4 = Model.NamedSelections.Children[3]
GCS = CS_GRP.Children[0]
LCS1 = CS_GRP.Children[1]

##### Section 4 Define remote point.

RMPT_GRP = Model.RemotePoints
RMPT_1 = RMPT_GRP.AddRemotePoint()
RMPT_1.Location = NS1
RMPT_1.XCoordinate=Quantity("7 [m]")
RMPT_1.YCoordinate=Quantity("0 [m]")
RMPT_1.ZCoordinate=Quantity("0 [m]")

#####  Section 5 Define Mesh Settings.

MSH = Model.Mesh
MSH.ElementSize =Quantity("0.5 [m]")
MSH.GenerateMesh()

#####  Section 6 Define boundary conditions.

##### Insert FIXED Support
FIX_SUP = STAT_STRUC.AddFixedSupport()
FIX_SUP.Location = NS2

##### Insert Frictionless Support
FRIC_SUP = STAT_STRUC.AddFrictionlessSupport()
FRIC_SUP.Location = NS3

#####  Section 7 Define remote force.

REM_FRC1 = STAT_STRUC.AddRemoteForce()
REM_FRC1.Location = RMPT_1
REM_FRC1.DefineBy =LoadDefineBy.Components
REM_FRC1.XComponent.Output.DiscreteValues = [Quantity("1e10 [N]")]

#####  Section 9 Insert directional deformation.

# DIR_DEF = STAT_STRUC.Solution.AddDirectionalDeformation()
# DIR_DEF.Location = NS1
# DIR_DEF.NormalOrientation =NormalOrientationType.XAxis

##### Section 10 Add Total Deformation and force reaction probe

TOT_DEF = STAT_STRUC.Solution.AddTotalDeformation()

##### Add Force Reaction
FRC_REAC_PROBE = STAT_STRUC.Solution.AddForceReaction()
FRC_REAC_PROBE.BoundaryConditionSelection = FIX_SUP
FRC_REAC_PROBE.ResultSelection =ProbeDisplayFilter.XAxis

##### Section 11 Solve and get the results.

##### Solve Static Analysis
STAT_STRUC.Solution.Solve(True)

dir_deformation_details = {
"Minimum": str(DIR_DEF.Minimum),
"Maximum": str(DIR_DEF.Maximum),
"Average": str(DIR_DEF.Average),
}

json.dumps(dir_deformation_details)

'''NOTE : All workflows will not be recorded, as recording is under development.'''

#region UI Action
with Transaction(True):
    body_20.Material = "0b11100b-3877-44f6-9845-9e3aee687bed"
#endregion


#region Details View Action
body_29.StiffnessBehavior = StiffnessBehavior.Rigid
#endregion

#region Context Menu Action
initial_conditions_46 = DataModel.GetObjectById(46)
initial_condition_56 = initial_conditions_46.InsertVelocity()
#endregion

#region Details View Action
selection = ExtAPI.SelectionManager.CreateSelectionInfo(SelectionTypeEnum.GeometryEntities)
selection.Ids = [6970]
initial_condition_56.Location = selection
#endregion

#region Details View Action
initial_condition_56.DefineBy = LoadDefineBy.Components
#endregion

#region Details View Action
initial_condition_56.XComponent = Quantity(-400000, "")
#endregion

#region Context Menu Action
solution_42 = DataModel.GetObjectById(42)
equivalent_plastic_strain_rst58 = solution_42.AddEquivalentPlasticStrainRST()
#endregion

#region Context Menu Action
equivalent_stress_59 = solution_42.AddEquivalentStress()
#endregion

#region Context Menu Action
total_deformation_60 = solution_42.AddTotalDeformation()
#endregion

#region Details View Action
selection = ExtAPI.SelectionManager.CreateSelectionInfo(SelectionTypeEnum.GeometryEntities)
selection.Ids = [6912]
total_deformation_60.Location = selection
#endregion

#region Context Menu Action
velocity_probe_62 = solution_42.AddVelocityProbe()
#endregion

#region Details View Action
selection = ExtAPI.SelectionManager.CreateSelectionInfo(SelectionTypeEnum.GeometryEntities)
selection.Ids = [6970]
velocity_probe_62.GeometryLocation = selection
#endregion

#region Details View Action
velocity_probe_62.ResultSelection = ProbeDisplayFilter.XAxis
#endregion

#region Context Menu Action
analysis_41 = DataModel.GetObjectById(41)
fixed_support_65 = analysis_41.AddFixedSupport()
#endregion

#region Details View Action
selection = ExtAPI.SelectionManager.CreateSelectionInfo(SelectionTypeEnum.GeometryEntities)
selection.Ids = [6907, 6908, 6909, 6906]
fixed_support_65.Location = selection
#endregion

#region Context Menu Action
fixed_support_67 = analysis_41.AddFixedSupport()
#endregion

#region Details View Action
selection = ExtAPI.SelectionManager.CreateSelectionInfo(SelectionTypeEnum.GeometryEntities)
selection.Ids = [33, 30, 31, 32]
fixed_support_67.Location = selection
#endregion

#region Details View Action
mesh_15 = Model.Mesh
mesh_15.ElementSize = Quantity(1, "mm")
#endregion

"""
)
print(output)

In [12]:
mechanical.exit()